In [15]:
!pip install xgboost
!pip install eli5

     --------------------------------------- 89.1/89.1 MB 17.7 MB/s eta 0:00:00
     -------------------------------------- 216.2/216.2 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 133.1/133.1 kB 7.7 MB/s eta 0:00:00
     ---------------------------------------- 47.0/47.0 kB ? eta 0:00:00
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107728 sha256=1522fa0c56bffc1dea4e3f03923d6d2fa369f311793efa358f7498fb21bf5e2f
  Stored in directory: c:\users\intel win11pro(12th)\appdata\local\pip\cache\wheels\7b\26\a5\8460416695a992a2966b41caa5338e5e7fcea98c9d032d055c
Successfully built eli5
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-project 0.11.1 requires ruamel-yaml, which is not installed.


In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error

import eli5
from eli5.sklearn import PermutationImportance



In [3]:
df = pd.read_csv('train.csv')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           4600 non-null   object 
 1   price          4600 non-null   float64
 2   bedrooms       4600 non-null   float64
 3   bathrooms      4600 non-null   float64
 4   sqft_living    4600 non-null   int64  
 5   sqft_lot       4600 non-null   int64  
 6   floors         4600 non-null   float64
 7   waterfront     4600 non-null   int64  
 8   view           4600 non-null   int64  
 9   condition      4600 non-null   int64  
 10  sqft_above     4600 non-null   int64  
 11  sqft_basement  4600 non-null   int64  
 12  yr_built       4600 non-null   int64  
 13  yr_renovated   4600 non-null   int64  
 14  street         4600 non-null   object 
 15  city           4600 non-null   object 
 16  statezip       4600 non-null   object 
 17  country        4600 non-null   object 
dtypes: float

In [11]:
# 숫자형 데이터만 입력
numeric_df = df.select_dtypes(include = 'number')

In [13]:
numeric_df

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated
0,3.130000e+05,3.0,1.50,1340,7912,1.5,0,0,3,1340,0,1955,2005
1,2.384000e+06,5.0,2.50,3650,9050,2.0,0,4,5,3370,280,1921,0
2,3.420000e+05,3.0,2.00,1930,11947,1.0,0,0,4,1930,0,1966,0
3,4.200000e+05,3.0,2.25,2000,8030,1.0,0,0,4,1000,1000,1963,0
4,5.500000e+05,4.0,2.50,1940,10500,1.0,0,0,4,1140,800,1976,1992
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4595,3.081667e+05,3.0,1.75,1510,6360,1.0,0,0,4,1510,0,1954,1979
4596,5.343333e+05,3.0,2.50,1460,7573,2.0,0,0,3,1460,0,1983,2009
4597,4.169042e+05,3.0,2.50,3010,7014,2.0,0,0,3,3010,0,2009,0
4598,2.034000e+05,4.0,2.00,2090,6630,1.0,0,0,3,1070,1020,1974,0


In [14]:
y_train = numeric_df['price']
numeric_df.pop('price')
X_train = numeric_df

In [28]:
model = RandomForestRegressor()
rf = model.fit(X_train,y_train)
output2 = model.predict(X_train)

print( '결정계수: ', model.score(X_train, y_train))
output2

결정계수:  0.8785995483775666


array([ 344796.5       , 1978224.61111111,  356403.71428572, ...,
        504316.79166687,  288177.29428571,  238024.        ])

In [29]:
model1 = XGBRegressor()
xgb = model1.fit(X_train,y_train)
output1 = model1.predict(X_train)

print( '결정계수: ', model1.score(X_train, y_train))
output1

결정계수:  0.9676704040907126


array([ 304639.97, 2339908.  ,  455172.5 , ...,  549264.94,  381116.03,
        292737.88], dtype=float32)

In [40]:
# feature_imporatance_rf = rf.feature_importances_
# feature_imporatance_xgb = xgb.feature_importances_

# estimator1 = lambda X: rf.predict(X.reshape(1, -1))
# estimator2 = lambda X: xgb.predict(X.reshape(1, -1))

In [50]:
cmap = plt.get_cmap('coolwarm')


a = eli5.show_weights(rf, feature_names = X_train.columns.tolist())
b = eli5.show_weights(xgb, feature_names = X_train.columns.tolist())
                  

In [51]:
a

Weight,Feature
0.3535 ± 0.3468,sqft_living
0.1635 ± 0.3212,yr_built
0.1242 ± 0.2858,sqft_lot
0.0856 ± 0.1785,sqft_above
0.0756 ± 0.2605,condition
0.0413 ± 0.1650,yr_renovated
0.0409 ± 0.1329,bathrooms
0.0389 ± 0.1053,sqft_basement
0.0270 ± 0.1652,floors
0.0250 ± 0.0927,bedrooms


In [45]:
eli5.show_weights(feature_imporatance_rf, feature_names=list('ABCDE'), cmap=cmap)
eli5.show_weights(feature_imporatance_xgb, feature_names=list('ABCDE'), cmap=cmap)

In [ ]:
cmap = plt.get_cmap('coolwarm')